In [70]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import os
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sklearn.metrics import roc_auc_score

bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/TRAINING-xgboost-dm'
# Define IAM role
role = get_execution_role()

## Retrieve the data from feature store


In [71]:
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

feature_group_name = "FG-New-data-flow-20250604-export-5dc2ccd6" 
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=feature_store_session)

In [72]:
# Build SQL query to features group
fs_query = feature_group.athena_query()
fs_table = fs_query.table_name
query_string = 'SELECT * FROM "'+fs_table+'"'
print('Running ' + query_string)

Running SELECT * FROM "fg_new_data_flow_20250604_export_5dc2ccd6_1749076781"


In [73]:
# Run Athena query. The output is loaded to a Pandas dataframe.
fs_query.run(query_string=query_string, output_location='s3://'+bucket+'/'+prefix+'/fs_query_results/')
fs_query.wait()
model_data = fs_query.as_dataframe()

INFO:sagemaker:Query a5f37098-bd34-4a90-adcf-ae7b626f8022 is being executed.
INFO:sagemaker:Query a5f37098-bd34-4a90-adcf-ae7b626f8022 successfully executed.


In [74]:
model_data

,custid,retained,esent,eopenrate,eclickrate,avgorder,ordfreq,paperless,refill,doorstep,favday,city,created_year,created_month,created_day,firstorder_year,firstorder_month,firstorder_day,lastorder_year,lastorder_month,lastorder_day,write_time,api_invocation_time,is_deleted
0,19394.0,1,16,0.000000,0.000000,9.23,0.000000,0,0,0,0.0,1.0,2011.0,4.0,1.0,2011.0,6.0,12.0,2011.0,6.0,12.0,2025-06-04 22:49:54.753,2025-06-04 22:44:58.000,False
1,24785.0,1,27,2.631441,2.104038,34.82,0.444078,1,1,0,0.0,0.0,2013.0,10.0,10.0,2013.0,11.0,1.0,2014.0,1.0,5.0,2025-06-04 22:49:54.753,2025-06-04 22:44:58.000,False
2,15390.0,1,35,0.289996,0.000000,46.38,0.033584,0,0,0,2.0,2.0,2010.0,7.0,9.0,2010.0,7.0,9.0,2012.0,2.0,2.0,2025-06-04 22:49:54.753,2025-06-04 22:44:58.000,False
3,7069.0,1,50,0.541325,0.000000,66.76,0.921980,0,0,0,4.0,0.0,2011.0,10.0,10.0,2011.0,10.0,10.0,2014.0,1.0,1.0,2025-06-04 22:49:54.753,2025-06-04 22:44:58.000,False
4,14775.0,0,2,3.383281,0.000000,51.23,0.000000,1,0,0,1.0,1.0,2013.0,5.0,24.0,2013.0,9.0,16.0,2013.0,9.0,16.0,2025-06-04 22:49:54.753,2025-06-04 22:44:58.000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30796,7437.0,0,0,0.000000,0.000000,396.59,0.000000,0,0,0,4.0,0.0,2012.0,11.0,20.0,2012.0,11.0,20.0,2012.0,11.0,20.0,2025-06-04 22:49:54.651,2025-06-04 22:45:08.000,False
30797,8741.0,0,0,0.000000,0.000000,32.91,0.000000,0,0,0,4.0,1.0,2012.0,10.0,3.0,2012.0,10.0,3.0,2012.0,10.0,3.0,2025-06-04 22:49:54.651,2025-06-04 22:45:08.000,False
30798,6294.0,1,28,1.449978,1.690745,38.46,0.549811,0,1,0,0.0,0.0,2013.0,10.0,18.0,2013.0,10.0,20.0,2013.0,11.0,24.0,2025-06-04 22:49:54.651,2025-06-04 22:45:08.000,False
30799,9204.0,1,40,2.199133,1.656930,68.23,0.150339,0,0,0,0.0,0.0,2011.0,10.0,2.0,2011.0,10.0,4.0,2013.0,11.0,10.0,2025-06-04 22:49:54.651,2025-06-04 22:45:08.000,False


In [75]:
# Prepare data SageMaker's XGBoost
# Amazon SageMaker's XGBoost container expects data in CSV data format.
# Note that the first column must be the target variable and the CSV should not include headers.

# move target (ie, retained) to first column
model_data2 = model_data.reindex(columns = ['retained', 'custid', 'esent', 'eopenrate', 'eclickrate',
                                         'avgorder', 'ordfreq', 'paperless',	'refill', 'doorstep', 
                                            'favday', 'city', 'created_year', 'created_month', 
                                            'created_day', 'firstorder_year', 'firstorder_month',
                                            'firstorder_day', 'lastorder_year', 'lastorder_month', 'lastorder_day'])
model_data2

,retained,custid,esent,eopenrate,eclickrate,avgorder,ordfreq,paperless,refill,doorstep,favday,city,created_year,created_month,created_day,firstorder_year,firstorder_month,firstorder_day,lastorder_year,lastorder_month,lastorder_day
0,1,19394.0,16,0.000000,0.000000,9.23,0.000000,0,0,0,0.0,1.0,2011.0,4.0,1.0,2011.0,6.0,12.0,2011.0,6.0,12.0
1,1,24785.0,27,2.631441,2.104038,34.82,0.444078,1,1,0,0.0,0.0,2013.0,10.0,10.0,2013.0,11.0,1.0,2014.0,1.0,5.0
2,1,15390.0,35,0.289996,0.000000,46.38,0.033584,0,0,0,2.0,2.0,2010.0,7.0,9.0,2010.0,7.0,9.0,2012.0,2.0,2.0
3,1,7069.0,50,0.541325,0.000000,66.76,0.921980,0,0,0,4.0,0.0,2011.0,10.0,10.0,2011.0,10.0,10.0,2014.0,1.0,1.0
4,0,14775.0,2,3.383281,0.000000,51.23,0.000000,1,0,0,1.0,1.0,2013.0,5.0,24.0,2013.0,9.0,16.0,2013.0,9.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30796,0,7437.0,0,0.000000,0.000000,396.59,0.000000,0,0,0,4.0,0.0,2012.0,11.0,20.0,2012.0,11.0,20.0,2012.0,11.0,20.0
30797,0,8741.0,0,0.000000,0.000000,32.91,0.000000,0,0,0,4.0,1.0,2012.0,10.0,3.0,2012.0,10.0,3.0,2012.0,10.0,3.0
30798,1,6294.0,28,1.449978,1.690745,38.46,0.549811,0,1,0,0.0,0.0,2013.0,10.0,18.0,2013.0,10.0,20.0,2013.0,11.0,24.0
30799,1,9204.0,40,2.199133,1.656930,68.23,0.150339,0,0,0,0.0,0.0,2011.0,10.0,2.0,2011.0,10.0,4.0,2013.0,11.0,10.0


In [76]:
# remove the headers
model_data2.columns = range(model_data2.shape[1])

model_data2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1,19394.0,16,0.000000,0.000000,9.23,0.000000,0,0,0,0.0,1.0,2011.0,4.0,1.0,2011.0,6.0,12.0,2011.0,6.0,12.0
1,1,24785.0,27,2.631441,2.104038,34.82,0.444078,1,1,0,0.0,0.0,2013.0,10.0,10.0,2013.0,11.0,1.0,2014.0,1.0,5.0
2,1,15390.0,35,0.289996,0.000000,46.38,0.033584,0,0,0,2.0,2.0,2010.0,7.0,9.0,2010.0,7.0,9.0,2012.0,2.0,2.0
3,1,7069.0,50,0.541325,0.000000,66.76,0.921980,0,0,0,4.0,0.0,2011.0,10.0,10.0,2011.0,10.0,10.0,2014.0,1.0,1.0
4,0,14775.0,2,3.383281,0.000000,51.23,0.000000,1,0,0,1.0,1.0,2013.0,5.0,24.0,2013.0,9.0,16.0,2013.0,9.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30796,0,7437.0,0,0.000000,0.000000,396.59,0.000000,0,0,0,4.0,0.0,2012.0,11.0,20.0,2012.0,11.0,20.0,2012.0,11.0,20.0
30797,0,8741.0,0,0.000000,0.000000,32.91,0.000000,0,0,0,4.0,1.0,2012.0,10.0,3.0,2012.0,10.0,3.0,2012.0,10.0,3.0
30798,1,6294.0,28,1.449978,1.690745,38.46,0.549811,0,1,0,0.0,0.0,2013.0,10.0,18.0,2013.0,10.0,20.0,2013.0,11.0,24.0
30799,1,9204.0,40,2.199133,1.656930,68.23,0.150339,0,0,0,0.0,0.0,2011.0,10.0,2.0,2011.0,10.0,4.0,2013.0,11.0,10.0


In [77]:
# separate data into train/test data split

# The model will be trained on 70% of data, it will then be evaluated on 20% of data to give us an estimate of the accuracy 
# we hope to have on "new" data, and 10% will be held back as a final testing dataset which will be used later on.


# Compute split indices
train_end = int(0.7 * len(model_data2))
val_end = int(0.9 * len(model_data2))

# Split the DataFrame into train, validation, and test sets
train_data = model_data2.iloc[:train_end]
validation_data = model_data2.iloc[train_end:val_end]
test_data = model_data2.iloc[val_end:]

In [78]:
# train data to CSV
train_data.to_csv('train.csv', index=False, header=False)
validation_data.to_csv('validation.csv', index=False, header=False)
test_data.to_csv('test.csv', index=False, header=False)

In [79]:
# copy data to S3 for SageMaker to access
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('validation.csv')

# Training the Model

In [80]:
# let's find the ECR container for the current location for Amazon SageMaker's implementation of XGBoost

container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [81]:
# Then, because we're training with the CSV file format, we'll create s3_inputs that our training function can use 
# as a pointer to the files in S3, which also specify that the content type is CSV.
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test/'.format(bucket, prefix), content_type='text/csv')

In [82]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m5.4xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5, #The maximum depth per tree. A deeper tree might increase the performance, 
                                     #but also the complexity and chances to overfit.
                        eta=0.2, #learning rate
                        gamma=4, #Gamma is a pseudo-regularisation parameter (Lagrangian multiplier), and depends on the other parameters. 
                                 #The higher Gamma is, the higher the regularization.
                        min_child_weight=6,
                        subsample=0.8, #Represents the fraction of observations to be sampled for each tree. 
                                       #A lower values prevent overfitting but might lead to under-fitting.
                        silent=0,
                        objective='reg:linear', #RMSE is the default metric for linear regression; 
                                                #The SageMaker XGBoost algorithm actually calculates RMSE and writes it to the 
                                                #CloudWatch logs on the data passed to the “validation” channel.
                        num_round=100 #number of trees
                       )

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation, 'test': s3_input_test}) 

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: xgboost-2025-06-05-02-32-10-571


2025-06-05 02:32:11 Starting - Starting the training job...
2025-06-05 02:32:45 Downloading - Downloading input data...
2025-06-05 02:33:01 Downloading - Downloading the training image...
2025-06-05 02:33:36 Training - Training image download completed. Training in progress...Arguments: train
[2025-06-05:02:33:57:INFO] Running standalone xgboost training.
[2025-06-05:02:33:57:INFO] File size need to be processed in the node: 3.09mb. Available memory size in the node: 55529.23mb
[2025-06-05:02:33:57:INFO] Determined delimiter of CSV input is ','
[02:33:57] S3DistributionType set as FullyReplicated
[02:33:57] 21560x20 matrix with 431185 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-06-05:02:33:57:INFO] Determined delimiter of CSV input is ','
[02:33:57] S3DistributionType set as FullyReplicated
[02:33:57] 6160x20 matrix with 123195 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[02:33:57] src/tree/updater

# AUC ROC